In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/scaling-and-smote/train_smote.pkl
/kaggle/input/scaling-and-smote/train_test_split.pkl


In [2]:
import joblib

X_train_smote = joblib.load('/kaggle/input/scaling-and-smote/train_smote.pkl')[0]
X_test = joblib.load('/kaggle/input/scaling-and-smote/train_test_split.pkl')[1]

In [3]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Define Autoencoder architecture
input_dim = X_train_smote.shape[1]
input_layer = Input(shape=(input_dim,))
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Train Autoencoder
autoencoder.fit(X_train_smote, X_train_smote, epochs=10, batch_size=32)

# Extract features using encoder part of Autoencoder
encoder = Model(input_layer, encoded)
autoencoder_features = encoder.predict(X_train_smote)
autoencoder_features_test = encoder.predict(X_test)

2025-04-20 14:05:14.154421: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745157914.339598      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745157914.399004      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1745157926.211275      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Epoch 1/10


I0000 00:00:1745157934.775305      93 service.cc:148] XLA service 0x7a3d5c00b6c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745157934.775725      93 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1745157935.050087      93 cuda_dnn.cc:529] Loaded cuDNN version 90300


  121/94934 ━━━━━━━━━━━━━━━━━━━━ 2:00 1ms/step - loss: 0.1072 

I0000 00:00:1745157935.816606      93 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


94934/94934 ━━━━━━━━━━━━━━━━━━━━ 121s 1ms/step - loss: 9.4759e-04
Epoch 2/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 117s 1ms/step - loss: 1.9540e-05
Epoch 3/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 117s 1ms/step - loss: 1.0993e-05
Epoch 4/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 117s 1ms/step - loss: 6.4640e-06
Epoch 5/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 118s 1ms/step - loss: 6.1831e-06
Epoch 6/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 118s 1ms/step - loss: 5.2833e-06
Epoch 7/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 117s 1ms/step - loss: 5.4576e-06
Epoch 8/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 119s 1ms/step - loss: 5.4899e-06
Epoch 9/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 117s 1ms/step - loss: 5.0310e-06
Epoch 10/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 117s 1ms/step - loss: 4.9969e-06
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 96s 1ms/step
13524/13524 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step


In [5]:
np.save('sae_features_train.npy', autoencoder_features)
np.save('sae_features_test.npy', autoencoder_features_test)
features = {'train': autoencoder_features, 'test': autoencoder_features_test}
np.save('sae_features.npy', features, allow_pickle=True)
autoencoder.save('autoencoder.h5')
encoder.save('encoder.h5')

In [7]:
from IPython.display import FileLink
FileLink(r'sae_features_train.npy')

/kaggle/working/sae_features_train.npy

In [8]:
from IPython.display import FileLink
FileLink(r'sae_features_test.npy')

/kaggle/working/sae_features_test.npy

In [9]:
from IPython.display import FileLink
FileLink(r'cnn_features.npy')

/kaggle/working/cnn_features.npy

In [10]:
from IPython.display import FileLink
FileLink(r'autoencoder.h5')

/kaggle/working/autoencoder.h5

In [11]:
from IPython.display import FileLink
FileLink(r'encoder.h5')

/kaggle/working/encoder.h5